In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# ==========================================
# 1. 가상의 HR 데이터 생성 (상황극 몰입)
# ==========================================
# 직원 1,000명, 퇴사율 약 15% 가정
X, y = make_classification(n_samples=1000, n_features=6, n_informative=4, 
                           n_redundant=0, weights=[0.85, 0.15], random_state=77)

cols_names = ['Satisfaction', 'Evaluation', 'Project_Count', 'Working_Hours', 'Year_At_Company', 'Promotion_Last_5Y']
df = pd.DataFrame(X, columns=cols_names)
df['Left'] = y

#print("=== 🏢 HR 데이터 샘플 ===")
#print(df.head())
#print("\n=== 타겟 비율 (0:잔류, 1:퇴사) ===")
#print(df['Left'].value_counts())

X_train, X_test, y_train, y_test = train_test_split(df.drop('Left', axis=1), df['Left'], test_size=0.2, random_state=77, stratify=df['Left'])

rf = RandomForestClassifier(random_state=77)
# 튜닝할 파라미터 조합
# class_weight: 데이터 불균형(퇴사자가 적음)을 해결하기 위해 모델이 '소수 클래스'에 가중치를 둘지 말지 결정
param_grid = {
    'n_estimators': [50, 100, 200], # 나무 개수
    'max_depth': [3, 5, 10 , None], # 나무 깊이
    'min_samples_leaf': [1, 2, 4], # 잎사귀에 최소 몇 개 데이터가 있어야 하는가? (과적합 방지)
    'class_weight': [None, 'balanced'] # 'balanced'를 쓰면 적은 데이터(퇴사자)를 더 중요하게 봄
}

# GridSearch 세팅 (5번 교차검증)
# scoring='f1': 단순히 많이 맞추는(accuracy) 것보다, 퇴사자(1)를 잘 찾는 조화평균(f1) 점수를 기준으로 1등을 뽑음
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='f1', verbose=1)

print("\n🚀 최적의 모델을 탐색 중입니다...")
grid_search.fit(X_train, y_train)

print("\n✅ GridSearchCV 결과")
print(f"Best Params: {grid_search.best_params_}")
print(f"Best F1 Score: {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_
final_pred = best_model.predict(X_test)

print("\n=== 📝 최종 성적표 (Test Set) ===")
print(classification_report(y_test, final_pred))


🚀 최적의 모델을 탐색 중입니다...
Fitting 5 folds for each of 72 candidates, totalling 360 fits

✅ GridSearchCV 결과
Best Params: {'class_weight': 'balanced', 'max_depth': 10, 'min_samples_leaf': 4, 'n_estimators': 50}
Best F1 Score: 0.6944

=== 📝 최종 성적표 (Test Set) ===
              precision    recall  f1-score   support

           0       0.92      0.93      0.93       169
           1       0.62      0.58      0.60        31

    accuracy                           0.88       200
   macro avg       0.77      0.76      0.76       200
weighted avg       0.88      0.88      0.88       200



`GridSearchCV`는 머신러닝에서 **가장 지루하지만 가장 중요한 '노가다'를 대신해주는 도구**입니다. 이걸 자유자재로 다루면 퇴근 시간이 빨라집니다.

이번에는 **[인사팀(HR) 직원 이탈 예측]** 시나리오로 가보겠습니다.
"우리 회사의 핵심 인재가 다음 달에 퇴사할 것인가?"를 맞추는 문제입니다.

---

### 🏢 실습 시나리오: "핵심 인재를 지켜라"

- **상황:** 당신은 대기업의 HR 데이터 분석가입니다.
- **데이터:** 직원의 만족도, 근무 시간, 프로젝트 수, 근속 연수 등의 데이터가 있습니다.
- **목표:** **GridSearchCV**를 사용하여, 퇴사할 것 같은 직원을 가장 잘 맞춰내는 **최적의 랜덤포레스트 옵션**을 찾아내세요.
- **새로운 도전:** 이번엔 `class_weight`라는 옵션을 튜닝에 추가해볼 겁니다. (데이터 불균형을 모델이 알아서 조절하게 하는 파라미터입니다.)

---

### 💡 이번 실습에서 주목할 점 (배움 포인트)

1.  **`scoring='f1'`**:

    - 이전에는 그냥 `scoring`을 지정 안 했죠? 그럼 기본값인 `accuracy(정확도)`로 1등을 뽑습니다.
    - 하지만 HR 데이터나 암 환자 데이터처럼 \*\*"소수(1)를 찾는 게 중요한 경우"\*\*에는 **`f1`** 점수가 높은 파라미터를 1등으로 뽑으라고 지정하는 게 실력입니다.

2.  **`class_weight`**:

    - 이 파라미터는 불균형 데이터에서 아주 유용합니다.
    - `balanced`로 설정되면, 모델이 학습할 때 **"어? 퇴사자 데이터가 적네? 그럼 퇴사자를 맞췄을 때 점수를 더 크게 줘야지\!"** 하고 자동으로 가중치를 조절합니다.

---

### 🏃‍♂️ 마지막 스퍼트\!

이 코드를 실행해서 **어떤 파라미터 조합이 'Best'로 선정되는지** 확인해보세요.
특히 `class_weight`가 `balanced`로 선택되는지, `None`으로 선택되는지 보는 것도 재미있는 관전 포인트입니다.

